In [1]:
import os
import pandas as pd
import sqlite3
from sqlalchemy import create_engine 
engine = create_engine("sqlite:///target_data.db")

In [3]:
conn= sqlite3.connect("target_data.db")

In [6]:
for file in os.listdir('target'):
    print(file)
    

customers.csv
geolocation.csv
orders (1).csv
order_items.csv
order_reviews.csv
payments.csv
products.csv
sellers.csv
Target - Problem Statement.pdf


### **Ingestion Function**

In [5]:
def ingest_db(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)
for file in os.listdir('target'):
    if file.endswith('.csv'):
        path=os.path.join('target', file)
        df=pd.read_csv(path, encoding="latin1")
        print(df.shape)
        ingest_db(df, file[:-4], engine)

(99441, 5)
(1000163, 5)
(99441, 8)
(112650, 7)
(99224, 6)
(103886, 5)
(32951, 9)
(3095, 4)


In [10]:
tables = pd.read_sql_query("select * from sqlite_master where type='table'",conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,customers,customers,2,"CREATE TABLE customers (\n\tcustomer_id TEXT, ..."
1,table,geolocation,geolocation,2236,CREATE TABLE geolocation (\n\tgeolocation_zip_...
2,table,orders,orders,12876,"CREATE TABLE ""orders"" (\n\torder_id TEXT, \n\t..."
3,table,order_items,order_items,17408,"CREATE TABLE order_items (\n\torder_id TEXT, \..."
4,table,order_reviews,order_reviews,21443,CREATE TABLE order_reviews (\n\treview_id TEXT...
5,table,payments,payments,24058,"CREATE TABLE payments (\n\torder_id TEXT, \n\t..."
6,table,products,products,25634,"CREATE TABLE products (\n\tproduct_id TEXT, \n..."
7,table,sellers,sellers,26202,"CREATE TABLE sellers (\n\tseller_id TEXT, \n\t..."


In [9]:
conn.execute(
    'alter table "orders (1)" rename to orders'
)
conn.commit()

In [7]:
customers      = pd.read_sql("SELECT * FROM customers", conn)
orders         = pd.read_sql("SELECT * FROM orders", conn)
order_items    = pd.read_sql("SELECT * FROM order_items", conn)
order_reviews  = pd.read_sql("SELECT * FROM order_reviews", conn)
payments       = pd.read_sql("SELECT * FROM payments", conn)
products       = pd.read_sql("SELECT * FROM products", conn)
sellers        = pd.read_sql("SELECT * FROM sellers", conn)
geolocation    = pd.read_sql("SELECT * FROM geolocation", conn)

In [100]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                        object
order_delivered_carrier_date             object
order_delivered_customer_date            object
order_estimated_delivery_date            object
dtype: object

In [30]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

In [15]:
orders.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [16]:
customers.dtypes

customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

In [22]:
customers.isna().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

In [18]:
order_items.isna().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [19]:
order_reviews.dtypes

review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_creation_date       object
review_answer_timestamp    object
dtype: object

In [21]:
order_reviews.isna().sum()

review_id                      0
order_id                       0
review_score                   0
review_comment_title       87677
review_creation_date           0
review_answer_timestamp        0
dtype: int64

In [24]:
customers.head(10)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
6,fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,4534,sao paulo,SP
7,5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR
9,4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG


In [26]:
geolocation.head(5)

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


#### **Timestamp in which orders were placed**

In [38]:
conn.execute("""
    update orders
    set order_purchase_timestamp = datetime(order_purchase_timestamp)
""")
conn.commit()

In [40]:
pd.read_sql_query("""
    select min(order_purchase_timestamp) start_time,
    max(order_purchase_timestamp) end_time from orders""",conn
)

,start_time,end_time
0,2016-09-04 21:15:19,2018-10-17 17:30:18


#### **Details of state & cities of customers who ordered during this period**


In [46]:
pd.read_sql_query(""" select 
                    c.customer_city, c.customer_state
                    from orders o 
                    join customers c 
                    on o.customer_id=c.customer_id
                    where strftime('%Y', o.order_purchase_timestamp) = '2018' 
                    and strftime('%m',o.order_purchase_timestamp) between '01' and '03'
 """,conn)                   

,customer_city,customer_state
0,santo andre,SP
1,sao paulo,SP
2,rio de janeiro,RJ
3,sao paulo,SP
4,sao paulo,SP
...,...,...
21203,valinhos,SP
21204,belo horizonte,MG
21205,praia grande,SP
21206,japuiba,RJ


#### **Trends in no.of orders placed over the past years**

In [50]:
pd.read_sql_query("""
    SELECT
        strftime('%Y-%m', order_purchase_timestamp) AS year_month,
        COUNT(order_id) AS total_orders
    FROM orders
    GROUP BY year_month
    ORDER BY year_month;
""", conn)


,year_month,total_orders
0,2016-09,4
1,2016-10,324
2,2016-12,1
3,2017-01,800
4,2017-02,1780
5,2017-03,2682
6,2017-04,2404
7,2017-05,3700
8,2017-06,3245
9,2017-07,4026


Orders increased rapidly from early 2017, peaking in late 2017.
2018 maintained consistently high monthly order volumes, indicating sustained customer adoption.
Extremely low counts in Sep–Dec 2016 and Sep–Oct 2018 are likely due to incomplete data coverage and were excluded from trend conclusions.

### **Monthly Seasonality in Orders**

In [53]:
pd.read_sql_query(""" select 
                strftime('%m', order_purchase_timestamp) Month, 
                count(order_id) Total_orders
                from orders 
                group by strftime('%m', order_purchase_timestamp)""",conn)


,Month,Total_orders
0,01,8069
1,02,8508
2,03,9893
3,04,9343
4,05,10573
5,06,9412
6,07,10318
7,08,10843
8,09,4305
9,10,4959


Monthly aggregation of order shows seasonality, with peak volume of orders observed between march & august-especially in May
&  August  highest orders were placed. While the lower counts of orders were observed in early(Jan-Feb) & late(Sep-Dec) of the Year

#### **Frequency of orders during different times of day**

In [59]:
pd.read_sql_query(""" select 
                    case
                        when cast(strftime('%H', order_purchase_timestamp) as integer) between 0 and 6
                            then 'Dawn' 
                        when cast(strftime('%H', order_purchase_timestamp) as integer) between 7 and 12
                           then 'Morning'
                        when cast(strftime('%H', order_purchase_timestamp) as integer) between 13 and 18 
                            then 'Afternoon'
                        when cast(strftime('%H', order_purchase_timestamp) as integer) between  19 and 23
                                then 'Night' end as Time_of_day, 
                         count(*) Total_orders 
                         from orders
                         group by Time_of_day
                         order by Total_orders desc
                        
                
                """,conn)

,Time_of_day,Total_orders
0,Afternoon,38135
1,Night,28331
2,Morning,27733
3,Dawn,5242


 **Time -of-day** analysis shows that Brazilian customers mostly place orders during 
afternoon(13-18 hrs) & during night ours(19-23 hrs). Morning orders are also significant, 
while Dawn hours record the lowest order volume..indicating limited early-morning engagement

### **How did Orders evolve over time** MoM Analysis

In [63]:
pd.read_sql_query(""" select 
                strftime('%m', order_purchase_timestamp) Month,
                strftime('%Y', order_purchase_timestamp) Year,
                count(order_id) Total_orders
                from orders
                group by strftime('%m', order_purchase_timestamp),
                strftime('%Y', order_purchase_timestamp) 
                order by Year, Month""",conn)

,Month,Year,Total_orders
0,09,2016,4
1,10,2016,324
2,12,2016,1
3,01,2017,800
4,02,2017,1780
5,03,2017,2682
6,04,2017,2404
7,05,2017,3700
8,06,2017,3245
9,07,2017,4026


A year–month analysis reveals a strong growth phase during 2017, with monthly orders increasing consistently.
In 2018, order volumes remain high but relatively stable, suggesting market maturation. Low order counts in
late 2016 and late 2018 are likely due to partial data coverage.

### **Distribution of customers accross the states**

In [68]:
pd.read_sql_query(""" select 
                    customer_state,
                    count(customer_id) Total_customers 
                    from customers 
                    group by customer_state
                    order by Total_customers desc""",conn)

,customer_state,Total_customers
0,SP,41746
1,RJ,12852
2,MG,11635
3,RS,5466
4,PR,5045
5,SC,3637
6,BA,3380
7,DF,2140
8,ES,2033
9,GO,2020


#### **% increase in the cost of orders from year 2017 to 2018 between (Jan-Aug)**

In [84]:
pd.read_sql_query("""with cte1 as (select
                            strftime('%Y', o.order_purchase_timestamp) Year,
                            sum(p.payment_value) payments
                    from orders o join payments p on
                    o.order_id=  p.order_id	
                    where cast(strftime('%Y', o.order_purchase_timestamp) as integer) in (2017,2018)
                    and cast(strftime('%m', o.order_purchase_timestamp) as integer) between 1 and 8
                    group by strftime('%Y', o.order_purchase_timestamp)),

                    cte2 as(select
                            Year,
                            payments,
                            lead(payments)over(order by Year desc) Prev_yr_payment           
                    from cte1)
                    select 
                    round(((payments-Prev_yr_payment)/Prev_yr_payment)*100,2) as Growth_percent
                    from cte2
                    
                    """,conn)
                    

,Growth_percent
0,136.98
1,NaN


#### **Total & Average value of order price for each state**

In [88]:
pd.read_sql_query("""
                    select 
                    c.customer_state,
                    sum(p.payment_value) Total_order_price,
                    Avg(p.payment_value) Avg_order_price
                    from customers c join orders o  
                    on o.customer_id=c.customer_id
                    join payments p on 
                    o.order_id=p.order_id 
                    group by c.customer_state
                    order by Total_order_price desc""",conn)
                    
                    

,customer_state,Total_order_price,Avg_order_price
0,SP,5998226.96,137.504630
1,RJ,2144379.69,158.525888
2,MG,1872257.26,154.706434
3,RS,890898.54,157.180406
4,PR,811156.38,154.153626
5,SC,623086.43,165.979337
6,BA,616645.82,170.816017
7,DF,355141.08,161.134791
8,GO,350092.31,165.763404
9,ES,325967.55,154.706953


#### **Mean & sum of freight value of each state**

In [94]:
pd.read_sql_query("""
                    select 
                    c.customer_state,
                    sum(oi.freight_value) Total_freight_value,
                    avg(oi.freight_value) Avg_freight_value 
                    from customers c 
                    join orders o 
                    on o.customer_id = c.customer_id 
                    join order_items oi on oi.order_id=o.order_id
                    group by c.customer_state
                    order by Total_freight_value desc""",conn)

,customer_state,Total_freight_value,Avg_freight_value
0,SP,718723.07,15.147275
1,RJ,305589.31,20.960924
2,MG,270853.46,20.630167
3,RS,135522.74,21.735804
4,PR,117851.68,20.531652
5,BA,100156.68,26.363959
6,SC,89660.26,21.470369
7,PE,59449.66,32.917863
8,GO,53114.98,22.766815
9,DF,50625.50,21.041355


In [104]:
cols = [
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"
]

for col in cols:
    orders[col] = pd.to_datetime(orders[col], errors="coerce")

orders.to_sql("orders", conn, if_exists="replace", index=False)



99441

In [105]:
orders.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

#### **Delivery time Analysis**

In [4]:
pd.read_sql_query("""
    select order_id, 
    julianday(order_delivered_customer_date)-julianday(order_purchase_timestamp) as time_to_deliver,
    
    julianday(order_delivered_customer_date)-julianday(order_estimated_delivery_date) diff_estimated_delivery
    from orders""",conn
)

,order_id,time_to_deliver,diff_estimated_delivery
0,e481f51cbdc54678b7cc49136f2d6af7,8.436574,-7.107488
1,53cdb2fc8bc7dce0b6741e2150273451,13.782037,-5.355729
2,47770eb9100c2d0c44946d9cf07ec65d,9.394213,-17.245498
3,949d5b44dbf5de918fe9c16f97b45f8a,13.208750,-12.980069
4,ad21c59c0840e6cb83a9ceb5573f8159,2.873877,-9.238171
...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.218009,-10.369433
99437,63943bddc261676b46f01ca7ac2f7bd8,22.193727,-1.265324
99438,83c1379a015df1e13d02aae0204711ab,24.859421,-5.524803
99439,11c177c8e97725db2631073c19f07b62,17.086424,-20.018819


Majority of orders were delivered ahead of schedule, with negative delivery variance indicating early fulfillment.
This suggests conservative delivery estimates, potentially improving customer satisfaction but also indicating room to optimize promised delivery timelines.

#### **states with the highest & lowest average freight**

In [13]:
pd.read_sql_query("""WITH state_freight AS (
    SELECT
        c.customer_state,
        AVG(oi.freight_value) AS avg_freight_value
    FROM order_items oi
    JOIN orders o ON o.order_id = oi.order_id
    JOIN customers c ON c.customer_id = o.customer_id
    GROUP BY c.customer_state
)

SELECT *
FROM (
    SELECT *
    FROM state_freight
    ORDER BY avg_freight_value DESC
    LIMIT 5
)

UNION ALL

SELECT *
FROM (
    SELECT *
    FROM state_freight
    ORDER BY avg_freight_value ASC
    LIMIT 5
)""",conn)



,customer_state,avg_freight_value
0,RR,42.984423
1,PB,42.723804
2,RO,41.069712
3,AC,40.073370
4,PI,39.147970
5,SP,15.147275
6,PR,20.531652
7,MG,20.630167
8,RJ,20.960924
9,DF,21.041355


State-wise analysis of average freight value highlights clear regional disparities. States with higher average freight costs may reflect longer delivery distances or logistical inefficiencies, whereas states with lower averages indicate more optimized supply-chain access

#### **Top 5 states with highest & lowest Avg delivery time**

In [18]:
pd.read_sql_query("""with cte1 as(select 
                    c.customer_state, 
                    avg(julianday(o.order_delivered_customer_date) - julianday(o.order_purchase_timestamp)) as Avg_delivery_time 
                    from customers c 
                    join orders o 
                    on o.customer_id=c.customer_id
                    group by c.customer_state) 
                    
                   select * from (select 
                    customer_state,
                    Avg_delivery_time 
                    from cte1 
                    order by Avg_delivery_time desc
                    limit 5)t 

                    union all 
                    select * from(select customer_state,
                    Avg_delivery_time 
                    from cte1 
                    order by Avg_delivery_time 
                    limit 5)t
                    
                    """,conn)
                    


,customer_state,Avg_delivery_time
0,RR,29.387546
1,AP,27.185068
2,AM,26.425991
3,AL,24.543855
4,PA,23.772917
5,SP,8.761357
6,PR,11.991582
7,MG,12.010258
8,DF,12.967568
9,SC,14.959297


State-wise analysis of average delivery time reveals strong regional disparities. Orders delivered to northern states have average delivery times exceeding 25 days, whereas deliveries to southern and southeastern states typically arrive within 9–15 days. This suggests a clear geographic impact on logistics performance.

#### **Top 5** states where the order delivery is really fast as compared to the estimated date of delivery

In [21]:
pd.read_sql_query("""
                    select c.customer_state,
                    julianday(o.order_delivered_customer_date)-julianday(o.order_estimated_delivery_date) diff_estimated_delivery
                    from orders o 
                    join customers c on 
                    c.customer_id=o.customer_id 
                    where diff_estimated_delivery is not null
                    group by c.customer_state 
                    order by diff_estimated_delivery
                    limit 5""",conn)

,customer_state,diff_estimated_delivery
0,AM,-47.227373
1,RS,-31.410995
2,MG,-28.495926
3,AP,-27.108414
4,AC,-22.140683


Negative estimated-to-actual delivery differences indicate early deliveries. States such as AM, RS, and MG exhibit the largest negative gaps, meaning orders are delivered substantially earlier than promised, reflecting strong logistics performance or conservative delivery estimates in these regions.

#### **Analysis based on the payments**


#### **month on month no. of orders placed using different paymen types.**


In [29]:
pd.read_sql_query("""select 
                    p.payment_type,
                    strftime('%Y-%m', o.order_purchase_timestamp) Year_month, 
                    count(o.order_id) Total_orders 
                    from payments p 
                    join orders o 
                    on p.order_id=o.order_id
                    group by p.payment_type, Year_month
                    order by Total_orders desc limit 5""",conn)

                    

,payment_type,Year_month,Total_orders
0,credit_card,2017-11,5897
1,credit_card,2018-03,5691
2,credit_card,2018-01,5520
3,credit_card,2018-05,5497
4,credit_card,2018-04,5455


Month-on-month analysis shows that, alongside credit cards, UPI contributes a substantial and stable share of total orders. Although credit cards lead across all periods, UPI emerges as a strong alternative payment method with consistent customer adoption.

#### **orders placed on the basis of the payment installments that have been paid.**

In [33]:
pd.read_sql_query("""select 
                    payment_installments,
                    count(order_id) Total_orders 
                    from payments
                    group by payment_installments
                    order by Total_orders desc limit 5""",conn)

,payment_installments,Total_orders
0,1,52546
1,2,12413
2,3,10461
3,4,7098
4,10,5328


Analysis of payment installments shows a strong preference for single-installment payments, accounting for the majority of orders. While usage declines with higher installment counts, multi-installment options remain relevant, particularly for higher-value purchases.